In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Import our filtering and visualization modules
from filter_functions import *
from visualizations import *

# Load the dataset
df = load_dataset('scholarship_dataset.csv')

# Display basic dataset information
print(f"Loaded dataset with {len(df)} scholarships")
print("\nDataset columns:")
print(df.columns.tolist())
print("\nSample data:")
display(df.head())

# Create dropdown options from dataset
states = ['All'] + sorted(df['State'].unique().tolist())
castes = ['All'] + sorted(df['Eligibility_Caste'].unique().tolist())
genders = ['All'] + sorted(df['Eligibility_Gender'].unique().tolist())
batches = sorted(df['Batch'].unique().tolist())
categories = sorted(df['Category'].unique().tolist())
courses = sorted(df['Course_Applicable'].unique().tolist())

# Create interactive widgets
state_dropdown = widgets.SelectMultiple(
    options=states,
    value=['All'],
    description='State:',
    disabled=False
)

caste_dropdown = widgets.SelectMultiple(
    options=castes,
    value=['All'],
    description='Caste:',
    disabled=False
)

gender_dropdown = widgets.Dropdown(
    options=genders,
    value='All',
    description='Gender:',
    disabled=False
)

min_amount_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=df['Amount'].max(),
    step=5000,
    description='Min Amount:',
    disabled=False
)

max_amount_slider = widgets.IntSlider(
    value=df['Amount'].max(),
    min=0,
    max=df['Amount'].max(),
    step=5000,
    description='Max Amount:',
    disabled=False
)

batch_selection = widgets.SelectMultiple(
    options=batches,
    description='Batch:',
    disabled=False
)

category_selection = widgets.SelectMultiple(
    options=categories,
    description='Category:',
    disabled=False
)

course_selection = widgets.SelectMultiple(
    options=courses,
    description='Course:',
    disabled=False
)

min_percentage_slider = widgets.IntSlider(
    value=60,
    min=50,
    max=100,
    step=5,
    description='Your %:',
    disabled=False
)

active_only_checkbox = widgets.Checkbox(
    value=True,
    description='Active scholarships only',
    disabled=False
)

# Create filter button
filter_button = widgets.Button(
    description='Filter Scholarships',
    button_style='success',
    tooltip='Click to apply filters'
)

# Create visualization button
viz_button = widgets.Button(
    description='Generate Visualizations',
    button_style='info',
    tooltip='Click to generate charts'
)

# Create export button
export_button = widgets.Button(
    description='Export Results',
    button_style='warning',
    tooltip='Click to export filtered results'
)

# Display widgets in organized layout
print("\n🔍 Select Filters:")
display(widgets.HBox([state_dropdown, caste_dropdown]))
display(widgets.HBox([gender_dropdown, batch_selection]))
display(widgets.HBox([category_selection, course_selection]))
display(widgets.HBox([min_amount_slider, max_amount_slider]))
display(widgets.HBox([min_percentage_slider, active_only_checkbox]))
display(widgets.HBox([filter_button, viz_button, export_button]))

# Create output area for results
output = widgets.Output()
display(output)

# Filtered dataframe to store results
filtered_df = df.copy()

# Define button click handlers
def on_filter_button_clicked(b):
    global filtered_df
    with output:
        clear_output()
        print("Applying filters...")
        
        filtered_df = apply_all_filters(
            df,
            states=state_dropdown.value if 'All' not in state_dropdown.value else None,
            castes=caste_dropdown.value if 'All' not in caste_dropdown.value else None,
            gender=gender_dropdown.value,
            min_amount=min_amount_slider.value,
            max_amount=max_amount_slider.value,
            batches=batch_selection.value if batch_selection.value else None,
            categories=category_selection.value if category_selection.value else None,
            courses=course_selection.value if course_selection.value else None,
            min_percentage=min_percentage_slider.value,
            active_only=active_only_checkbox.value
        )
        
        print(f"Found {len(filtered_df)} matching scholarships")
        if len(filtered_df) > 0:
            display(filtered_df)
        else:
            print("No scholarships found matching your criteria. Try adjusting your filters.")

def on_viz_button_clicked(b):
    with output:
        clear_output()
        if len(filtered_df) > 0:
            print(f"Generating visualizations for {len(filtered_df)} scholarships...")
            generate_all_visualizations(filtered_df)
        else:
            print("No data to visualize. Please apply filters first.")

def on_export_button_clicked(b):
    with output:
        clear_output()
        if len(filtered_df) > 0:
            export_filename = 'filtered_scholarships.csv'
            filtered_df.to_csv(export_filename, index=False)
            print(f"✅ Exported {len(filtered_df)} scholarships to {export_filename}")
        else:
            print("No data to export. Please apply filters first.")

# Attach handlers to buttons
filter_button.on_click(on_filter_button_clicked)
viz_button.on_click(on_viz_button_clicked)
export_button.on_click(on_export_button_clicked)
